# **East Asia heatwave study - analysis of model data**

In [45]:
source("../methods_extRemes.r")
source("../wwa_nonstationary_fitting.r")

# list return periods to use in later analysis
return_periods <- list("ib-ext" = 10, "ib-trop" = 10, "tl" = 10)

## **CORDEX tasmax**

In [ ]:
source("../wwa_nonstationary_fitting.r")




# model_results <- function(df, dist, fittype, varnm, covnm_1, covnm_2 = NA, cov1_2022, cov1_hist, cov1_fut, cov2_2022 = 0, cov2_hist = 0, cov2_fut = 0,
#                                 lower = F, rp = 10, delta_relative = T, nsamp = 1000, seed = 1) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    df_eval <- df[df$year >= 1979 & df$year <= 2022,]
    df_attr <- df[df$year <= 2022,]
    df_proj <- df[df$year <= 2050,]
    
    mdl_eval <- fit_ns(dist, fittype, df_eval, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_attr <- fit_ns(dist, fittype, df_attr, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_proj <- fit_ns(dist, fittype, df_proj, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## EVALUATION (get model parameters)
    
    if(fittype == "fixeddisp") {
        disp_eval <- mdl_eval$par["sigma0"] / mdl_eval$par["mu0"]
        boot_eval <- sapply(1:nsamp, function(i) {
            
            boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
            boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
            boot_mdl$par["sigma0"] / boot_mdl$par["mu0"]
        })
        ci_eval <- setNames(c(disp_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("disp_est", "disp_lower", "disp_upper"))
    } else {
        var_eval <- mdl_eval$par["sigma0"]
        boot_eval <- sapply(1:nsamp, function(i) {
            
            boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
            boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
            boot_mdl$par["sigma0"]
        })
        ci_eval <- setNames(c(var_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("var_est", "var_lower", "var_upper"))
    }
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## ATTRIBUTION (get event value, PR & DI)
    
    rl_2022 <- map_from_u(1/rp, mdl_attr, cov1 = cov1_2022, cov2 = cov2_2022)
    if(substr(varnm,1,5) == "log10") { actual_rl2022 <- 10^rl_2022 } else { actual_rl2022 <- rl_2022 }
    
    pr_eval <- prob_ratio(mdl_attr, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
    dI_eval <- Delta_I(mdl_attr, rp, cov1 = cov1_2022, cov1_cf = cov1_hist, relative = delta_relative)
    
    boot_attr <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_attr[sample(1:nrow(df_attr), nrow(df_attr), replace = T),]
        boot_mdl <- fit_ns(mdl_attr$dist, mdl_attr$type, boot_df, varnm = mdl_attr$varnm, covnm_1 = mdl_attr$covnm_1, covnm_2 = mdl_attr$covnm_2, lower = mdl_attr$lower)
        
        boot_pr <- prob_ratio(boot_mdl, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_attr <- rbind("est" = c(pr_eval, dI_eval), apply(boot_attr, 1, quantile, c(0.025, 0.975)))
    ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))
                          
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## PROJECTION (get PR & DI)

    pr_proj <- prob_ratio(mdl_proj, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut)
    dI_proj <- Delta_I(mdl_proj, 10, cov1 = cov1_2022, cov1_cf = cov1_fut, relative = delta_relative)
                             
    boot_proj <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_proj[sample(1:nrow(df_proj), nrow(df_proj), replace = T),]
        boot_mdl <- fit_ns(mdl_proj$dist, mdl_proj$type, boot_df, varnm = mdl_proj$varnm, covnm_1 = mdl_proj$covnm_1, covnm_2 = mdl_proj$covnm_2, lower = mdl_proj$lower)
        
        boot_pr <- prob_ratio(boot_mdl, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_proj <- rbind("est" = c(pr_proj, dI_proj), apply(boot_proj, 1, quantile, c(0.025, 0.975)))
      
    # invert
    ci_proj[,"PR"] <- 1/ci_proj[c(1,3,2),"PR"]
    ci_proj[,"DI"] <- -ci_proj[c(1,3,2),"DI"]   
                             
    ci_proj <- unlist(lapply(colnames(ci_proj), function(cnm) setNames(ci_proj[,cnm], paste("proj", cnm, c("est", "lower", "upper"), sep = "_"))))
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                             
    # t(setNames(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr, ci_proj)), gsub(".dat", "", gsub(".+pr-mam_", "", fnm))))      
    t(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr, ci_proj)))     
# }

In [30]:
region <- "ib-ext"
fnm <- list.files("ts", pattern = paste0("tx4x-",region), full.names = T)[1]

In [95]:
ns_mdl <- fevd(tx4x, df[df$year >= 1979 & df$year <= 2022,], location.fun = ~gmst, type = "GEV", method = "MLE")

qcov_2022 <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"]))
qcov_hist <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"] - 1.2))
qcov_fut <- make.qcov(ns_mdl, vals = list("mu1" = df[df$year == 2022, "gmst"] + 0.8))

rp <- 10

seed <- 1
nsamp <- 5
lower = F

In [78]:
        set.seed(seed)
        
        # load necessary data
        gmst_fnm <- list.files("ts", paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T)
        gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
        
        ts <- load_ts(fnm)
        if(ncol(ts) == 3) {
            colnames(ts) <- c("year", "h", "tx4x")
        } else {
            colnames(ts) <- c("year", "tx4x")
        }
        df <- merge(gmst, ts)
        
        # covariates defining current, past & future climates
        gmst_2022 <- gmst[gmst$year == 2022, "gmst"]
        gmst_hist <- gmst_2022 - 1.2
        
        # fit models
        mdl_eval <- update(ns_mdl, data = df[df$year >= 1979 & df$year <= 2022,])
        mdl_attr <- update(ns_mdl, data = df[df$year <= 2022,])
        mdl_proj <- update(ns_mdl, data = df[df$year <= 2050,])     

In [68]:
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        ## EVALUATION (get model parameters)

        ci_eval <- setNames(suppressWarnings(ci.fevd(mdl_eval, type = "parameter", method = "boot"))["scale",],
                            c("var_lower", "var_est", "var_upper"))

In [113]:
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        ## ATTRIBUTION (get event value, PR & DI)

        rl_2022 <- return.level(mdl_attr, rp, qcov = qcov_2022)[1]

        pr_attr <- pextRemes(mdl_attr, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(mdl_attr, rl_2022, qcov = qcov_hist, lower.tail = lower)
        dI_attr <- rl_2022 - return.level(mdl_attr, rp, qcov = qcov_hist)[1]

        boot_attr <- sapply(1:nsamp, function(i) {

            boot_df <- mdl_attr$cov.data[sample(1:length(mdl_attr$x), length(mdl_attr$x), replace = T),]
            boot_mdl <- update(mdl_attr, data = boot_df)

            boot_pr <- pextRemes(boot_mdl, rl_2022, qcov = qcov_2022, lower.tail = lower) / pextRemes(boot_mdl, rl_2022, qcov = qcov_hist, lower.tail = lower)
            boot_di <- return.level(boot_mdl, rp, qcov = qcov_2022)[1] - return.level(boot_mdl, rp, qcov = qcov_hist)[1]

            setNames(c("PR" = boot_pr, "DI" = boot_di), c("PR", "DI"))
        })
        ci_attr <- rbind("est" = c(pr_attr, dI_attr), apply(boot_attr, 1, quantile, c(0.025, 0.975), skipna = T))
        # ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))

In [115]:
ci_attr

,threshold,
est,Inf,1.370527
2.5%,Inf,1.352430
97.5%,Inf,1.580554


In [99]:
?sample

sample {base},R Documentation
x,"either a vector of one or more elements from which to choose, or a positive integer. See ‘Details.’"
n,"a positive number, the number of items to choose from. See ‘Details.’"
size,a non-negative integer giving the number of items to choose.
replace,should sampling be with replacement?
prob,a vector of probability weights for obtaining the elements of the vector being sampled.
useHash,"logical indicating if the hash-version of the algorithm should be used. Can only be used for replace = FALSE, prob = NULL, and size <= n/2, and really should be used for large n, as useHash=FALSE will use memory proportional to n."


In [ ]:
# fit model & get results for each season
invisible(sapply(c("ib-ext", "ib-trop", "tl"), function(seas) {
    
    rp <- return_periods[[seas]]                  # return period depending on variable
    
    # loop over all available models, get results for each
    res <- sapply(list.files("ts", pattern = paste0("tx4x-",seas), full.names = T), function(fnm) {

        # load necessary data
        gmst_fnm <- list.files("ts", paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T)
        gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
        
        ts <- load_ts(fnm)
        if(ncol(ts) == 3) {
            col.names(ts) <- c("year", "h", "tx4x")
        } else {
            col.names(ts) <- c("year", "tx4x")
        }
        df <- merge(gmst, ts)
        
        # covariates defining current, past & future climates
        gmst_2022 <- gmst[gmst$year == 2022, "gmst"]
        gmst_hist <- gmst_2022 - 1.2
        
        # fit model & get results
        nsgev_fit <- fevd(tx4x, df, location.fun = ~gmst, type = "GEV", method = "MLE")
        data.frame(model_results(df, "norm", "fixeddisp", varnm = "log10pr", covnm_1 = "gmst", covnm_2 = covnm_2, lower = T, rp = rp, delta_relative = T, nsamp = 1000,
                                       cov1_2022 = gmst_2022, cov1_hist = gmst_hist, cov1_fut = gmst_fut, 
                                       cov2_2022 = nino_2022, cov2_hist = nino_hist, cov2_fut = nino_fut))
    }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(paste0(".+pr-",seas,"_"), "", rownames(res)))
    write.csv(res, paste0("res/EA-drought_pr-",seas,"_cordex_fitted.csv"))
}))